萧何

In [ ]:
XH = ["All in One"]
XH = ["9"]
#XH = ["100", "200"]
Cruz = 15
# region
import os
import nbformat
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import ttk
from tkinter.scrolledtext import ScrolledText
import re
import random

class HX:
    
    def __init__(self, root, markdown_content):
        self.root = root
        self.root.title("Markdown Quiz")
        self.root.geometry("800x600")
        self.HEADER_OPTIONS = XH
        # 挖空内容的变量
        self.markdown_content = markdown_content
        self.lines = self.markdown_content.split("\n")
        self.correct_streak = 0
        # 获取所有标题并初始化下拉菜单的选项

        self.terminal_content = self.find_terminal_headers_with_content(self.lines)
        
        # 当前正确答案的内容
        self.correct_answer = ""
        self.current_option = ""

        # 设置显示和控制区域
        self.create_display_area()
        self.create_option_area()

        # 绑定按键
        self.root.bind("<space>", lambda event: self.show_random_options())
        self.root.bind("<Return>", lambda event: self.check_answer())

        # 显示初始的挖空内容和选项
        self.show_new_quiz()

    def find_terminal_headers_with_content(self, lines, selected_headers=None):
        # 提取符合条件的标题内容
        terminal_headers = []
        for i, line in enumerate(lines):
            # 检查当前行是否是标题行
            match = re.match(r"^(#+)\s", line)
            if match:
                current_level = len(match.group(1))  # 获取标题层级
                content = []

                # 收集当前标题下的内容
                for j in range(i + 1, len(lines)):
                    next_match = re.match(r"^(#+)\s", lines[j])
                    if next_match:  # 如果遇到新的标题，停止收集内容
                        break
                    content.append(lines[j])

                content_str = "\n".join(content).strip()  # 合并内容
                terminal_headers.append((line, content_str))  # 添加标题和内容到结果
        
        if self.HEADER_OPTIONS:
            selected_headers = random.choice(self.HEADER_OPTIONS)
        
        # 如果指定了 selected_headers，则进行筛选
        if selected_headers:
            terminal_headers = [
            (header, content) for header, content in terminal_headers
            if re.match(rf"^#\s*{re.escape(selected_headers)}(?:\s|$)", header)
        ]

            # 处理 header 和 content
            if terminal_headers:
                terminal_headers = [
                    (
                        # 处理 header：删除 '#'，加上 '-'
                        header.lstrip("#").strip() + "\n",
                        # 在 content 开头添加修改后的 header 内容，第一条内容前加两个空格
                        f"- {header.lstrip('#').strip()}\n  {content}"
                    )
                    for header, content in terminal_headers
                ]

        return terminal_headers

    def create_display_area(self):
        # 创建展示Markdown内容的区域，占窗口一半高度
        display_frame = tk.Frame(self.root)
        display_frame.pack(side=tk.TOP, expand=True, fill=tk.BOTH, padx=10, pady=10)
        display_frame.config(height=300)

        self.text_area = ScrolledText(display_frame, wrap=tk.WORD, font=("Helvetica", 16))
        self.text_area.pack(expand=True, fill=tk.BOTH)
        self.text_area.config(state=tk.DISABLED)

    def display_image(self, image_path, max_size=(300, 300)):
        # Enable the text area for modification
        self.text_area.config(state=tk.NORMAL)

        # Load the image
        img = Image.open(image_path)
        original_width, original_height = img.size

        # Calculate scaling factor
        max_width, max_height = max_size
        scale_factor = min(max_width / original_width, max_height / original_height, 1)

        # Compute new dimensions
        new_width = int(original_width * scale_factor)
        new_height = int(original_height * scale_factor)

        # Resize the image with high-quality resampling
        img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)

        # Convert to PhotoImage for Tkinter
        photo = ImageTk.PhotoImage(img)

        # Create a label to hold the image
        image_label = tk.Label(self.text_area, image=photo, borderwidth=0)
        image_label.photo = photo  # Retain a reference to prevent garbage collection

        # Insert the label as a window in the text widget
        self.text_area.window_create(tk.END, window=image_label)
        self.text_area.insert(tk.END, " ")  # Add a space between images

        # Disable the text area to prevent editing
        self.text_area.config(state=tk.DISABLED)

    def display_a(self, image_path, max_size=(300, 300)):
        # Enable the text area for modification
        self.text_area.config(state=tk.NORMAL)

        # Load the image
        img = Image.open(image_path)
        original_width, original_height = img.size

        # Calculate scaling factor
        max_width, max_height = max_size
        scale_factor = min(max_width / original_width, max_height / original_height, 1)

        # Compute new dimensions
        new_width = int(original_width * scale_factor)
        new_height = int(original_height * scale_factor)

        # Resize the image with high-quality resampling
        img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)

        # Convert to PhotoImage for Tkinter
        photo = ImageTk.PhotoImage(img)

        # Create a frame to hold the image with a black border
        frame = tk.Frame(self.text_area, bg="black", bd=1)  # Black border with 2-pixel width
        image_label = tk.Label(frame, image=photo, bg="black")  # Label inside the frame
        image_label.photo = photo  # Retain a reference to prevent garbage collection
        image_label.pack(padx=1, pady=1)  # Add padding inside the frame

        # Insert the frame as a window in the text widget
        self.text_area.window_create(tk.END, window=frame)
        self.text_area.insert(tk.END, " ")  # Add a space between images

        # Disable the text area to prevent editing
        self.text_area.config(state=tk.DISABLED)

    def create_option_area(self):
        # 创建选项和按钮区域，占窗口另一半高度
        option_frame = tk.Frame(self.root)
        option_frame.pack(side=tk.TOP, fill=tk.BOTH, expand=True, padx=10, pady=10)

        # 创建选项标签，允许换行显示
        self.option_label = tk.Label(option_frame, text="", font=("Helvetica", 18), width=80, wraplength=600, anchor="center")
        self.option_label.pack(pady=10)

        # 创建按钮容器并固定位置
        button_frame = tk.Frame(option_frame)
        button_frame.pack(pady=10)

        self.check_button = tk.Button(
            button_frame,
            text="Check",
            command=self.check_answer,
            width=10,  # 调整按钮的宽度
            height=2,  # 调整按钮的高度
            font=("Helvetica", 14)  # 调整字体大小
        )
        self.check_button.pack(side=tk.LEFT, padx=20)

        self.next_button = tk.Button(
            button_frame,
            text="Next",
            command=self.show_random_options,
            width=10,  # 调整按钮的宽度
            height=2,  # 调整按钮的高度
            font=("Helvetica", 14)  # 调整字体大小
        )
        self.next_button.pack(side=tk.LEFT, padx=20)

    def show_new_quiz(self):
        # 显示新的挖空内容和正确答案
        self.correct_streak += 1
        if self.terminal_content:
            # 随机选择一个最末端标题及其内容
            random_terminal_header, content = random.choice(self.terminal_content)
            blanked_content, correct_answer = self.find_and_blank_lowest_content(content)

            
            display_content = f"\n\n{blanked_content}"  # 包含标题和挖空后的内容
            self.render_markdown_to_text(display_content)
            self.correct_answer = correct_answer.strip("- ").strip()
            self.show_random_options()
            # 将标题和内容合并到显示内容中
            correct_answer = correct_answer.lstrip("- ").lstrip()
            print(correct_answer)
            
            entries = [line.lstrip('- ').strip() for line in blanked_content.strip().split('\n') if line.strip()]
            # Add the correct_answer as the last entry
            #entries.append(correct_answer.strip())

            # Generate sanitized paths for each entry
            image_paths = []  # Collect valid image paths here
            image_directory = r"D:\desktop\Pic"
            for entry in entries:
                # Sanitize the entry to create a valid file name
                sanitized_entry = entry.replace(":", "_").replace("/", "_")
                image_path = os.path.join(image_directory, f"{sanitized_entry}.png")
                
                # Check if the image file exists
                if os.path.exists(image_path):
                    image_paths.append(image_path)  # Add valid paths to the list

            # Display all collected images
            for path in image_paths:
                self.display_image(path)  # Assuming self.display_image handles displaying the image
            
            image_a = os.path.join(image_directory, correct_answer + ".png")
            # Display the image if it exists
            if os.path.exists(image_a):
                self.display_a(image_a)

    def show_random_options(self):
        # 获取当前正确答案和三个干扰选项，并随机排列
        if self.correct_answer:
            # 将正确答案和干扰选项组合在一起并打乱顺序
            options = [self.correct_answer] + self.get_random_disturbing_options(num_options=3)
            random.shuffle(options)  # 随机排列选项顺序
            
            # 随机选择其中一个选项显示
            self.current_option = random.choice(options)
            self.option_label.config(text=self.current_option)

    def check_answer(self):
        # 检查当前选项是否正确
        global selected_headers
        if self.current_option == self.correct_answer:
            #self.terminal_content = self.update_terminal_content(self.lines)
            if self.correct_streak >= Cruz:
                root.destroy()
            else:
                selected_headers = random.choice(self.HEADER_OPTIONS)
                self.terminal_content = self.find_terminal_headers_with_content(self.lines)
                self.show_new_quiz()
        else:
            self.correct_streak = 0

    def render_markdown_to_text(self, content):
        # 在显示区域渲染 Markdown 内容
        self.text_area.config(state=tk.NORMAL)
        self.text_area.delete("1.0", tk.END)
        lines = content.split("\n")

        for line in lines:
            if line.startswith("- "):
                # 列表项
                self.text_area.insert(tk.END, line + "\n")
            else:
                # 普通文本
                self.text_area.insert(tk.END, line + "\n")  # 普通文本无样式

        self.text_area.config(state=tk.DISABLED)

    def find_and_blank_lowest_content(self, content):
        lines = [line for line in content.split("\n") if line.strip()]  # 去掉空行
        non_lowest_items = []
        def get_indent_level(line):
            """计算行的缩进数量（以空格为单位）"""
            indent_level = len(line) - len(line.lstrip())  # 计算缩进级别
            return indent_level

        # 遍历所有行
        for i, line in enumerate(lines):
            if re.match(r"^\s*-\s", line):  # 条目以 - 开头
                current_indent = get_indent_level(line)  # 当前条目的缩进层级
                has_direct_child = False
                
                # 检查后续行，判断是否有直接子项
                for j in range(i + 1, len(lines)):
                    next_line = lines[j]
                    next_indent = get_indent_level(next_line)  # 后续条目的缩进层级
                    
                    if next_indent > current_indent:  # 是子项
                        has_direct_child = True
                        break
                    if next_indent <= current_indent:  # 遇到同级或上级条目，停止检查
                        break

                if has_direct_child:
                    non_lowest_items.append(i)

        if non_lowest_items:
            # 随机选择一个非最低条目
            chosen_index = random.choice(non_lowest_items)
            chosen_line = lines[chosen_index]

            # 找到其直接子项
            child_lines = []
            current_indent = get_indent_level(lines[chosen_index])  # 被选条目的缩进层级
            for j in range(chosen_index + 1, len(lines)):
                next_line = lines[j]
                next_indent = get_indent_level(next_line)

                if next_indent == current_indent + 2:  # 直接子项（假设每一级缩进为2个空格）
                    child_lines.append(next_line)
                if next_indent <= current_indent:  # 遇到同级或上级条目，停止寻找子项
                    break

            # 随机选择一个子项，如果没有子项则设为 None
            chosen_child = random.choice(child_lines) if child_lines else None

            # 向上检索找到所有所属的父级条目，直到 indent_level 为 0
            parent_lines = []
            for k in range(chosen_index - 1, -1, -1):  # 从当前条目向上检索
                prev_line = lines[k]
                prev_indent = get_indent_level(prev_line)
                if prev_indent < current_indent and prev_indent % 2 == 0:  # 找到更高级别的条目
                    parent_lines.append(prev_line)
                    current_indent = prev_indent  # 更新当前缩进级别
                if prev_indent == 0:  # 到顶层条目时停止
                    break

            parent_lines.reverse()  # 保证从顶层条目到当前条目按顺序排列

            # 构造挖空后的内容，包括父级条目、chosen_line 和其子项
            blanked_content = []
            for line in parent_lines:
                blanked_content.append(line)  # 添加所有父级条目
            if chosen_line:
                blanked_content.append(chosen_line)
            for line in child_lines:
                if line == chosen_child:
                    # 只挖空随机选择的子项
                    blanked_content.append(re.sub(r"- .+", "- " + "_" * (len(line) - 2), line))
                else:
                    blanked_content.append(line)
            correct_answer = chosen_child.strip() if chosen_child else None
            
            return "\n".join(blanked_content), correct_answer
        else:
            # 当没有非最低条目时，直接从所有条目中随机挖空
            all_items = [line for line in lines if re.match(r"^\s*-\s", line)]  # 所有符合条目格式的行
            chosen_item = random.choice(all_items) if all_items else None

            # 构造挖空后的内容
            blanked_content = []
            for line in lines:
                if line == chosen_item:
                    # 挖空选中的条目
                    blanked_content.append(re.sub(r"- .+", "- " + "_" * (len(line) - 2), line))
                else:
                    blanked_content.append(line)
        
            # 返回挖空后的内容和答案
            correct_answer = chosen_item.strip() if chosen_item else None
            return "\n".join(blanked_content), correct_answer

    def get_random_disturbing_options(self, num_options=3):
        # 遍历 self.lines，选择最低层级的项目符号行
        all_lowest_options = [
        line.lstrip("- ").lstrip()
        for _, content in self.terminal_content  # 遍历元组，content是第二部分
        for line in content.split("\n")  # 按换行符分割每部分内容
        if re.match(r"^\s*-\s", line)  # 筛选以"- "开头的行
        ]
        # 过滤掉正确答案
        filtered_options = [option for option in all_lowest_options if option != self.correct_answer]
        # 随机选择 num_options 个干扰选项
        disturbing_options = random.sample(filtered_options, num_options)
        return disturbing_options

if __name__ == "__main__":
    # 提取.ipynb文件中的Markdown内容
    filepath = r"D:\desktop\A.ipynb"  # 请调整为您的文件路径
    def extract_markdown_from_ipynb(filepath):
        # 从 .ipynb 文件中提取 Markdown 内容
        with open(filepath, "r", encoding="utf-8") as file:
            notebook = nbformat.read(file, as_version=4)
        markdown_cells = [cell['source'] for cell in notebook.cells if cell['cell_type'] == 'markdown']
        return "\n\n---\n\n".join(markdown_cells)
    markdown_content = extract_markdown_from_ipynb(filepath)

    # 创建并启动Quiz应用
    root = tk.Tk()
    app = HX(root, markdown_content)
    root.mainloop()
# endregion

Amino

In [22]:
# region
import os
import sqlite3
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import random

# 连接到数据库并读取数据
def load_drug_data(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    #cursor.execute("SELECT Q, pathway, class FROM Pic WHERE class = 'Amino_1' or class = 'Amino_2' or class = 'Amino_3'")
    cursor.execute("SELECT Q, pathway, class FROM Pic WHERE class = 'Amino_3'")
    data = cursor.fetchall()
    conn.close()
    return data

def create_table(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Drop the table if it exists
    cursor.execute('''
    DROP TABLE IF EXISTS Pic
    ''')
    
    # Create the table
    cursor.execute('''
    CREATE TABLE Pic (
        Q TEXT PRIMARY KEY,
        pathway TEXT,
        class TEXT
    )
    ''')
    
    conn.commit()
    conn.close()

# 假设数据库路径为 d:\\desktop\\Mein Kampf\\Guerrilla.db
db_path = r'd:\desktop\Mein Kampf\Guerrilla.db'
create_table(db_path)
data = load_drug_data(db_path)

# 初始化药物结构字典，包含 class 信息
drugs_structures = {}
for row in data:
    q, pathway, class_name = row
    if class_name not in drugs_structures:
        drugs_structures[class_name] = {}
    drugs_structures[class_name][q] = pathway

# 初始化连续答对的计数器
correct_streak = 0
target_streak = 20

def insert_data(db_path, q, pathway, class_name):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute('''
    INSERT OR REPLACE INTO Pic (Q, pathway, class)
    VALUES (?, ?, ?)
    ''', (q, pathway, class_name))
    conn.commit()
    conn.close()

def read_folder_and_insert_data(base_folder, db_path):
    for class_name in os.listdir(base_folder):
        class_path = os.path.join(base_folder, class_name)
        if os.path.isdir(class_path):
            for file_name in os.listdir(class_path):
                file_path = os.path.join(class_path, file_name)
                if os.path.isfile(file_path) and file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                    q, _ = os.path.splitext(file_name)  # 去除文件后缀
                    pathway = file_path
                    insert_data(db_path, q, pathway, class_name)

def generate_question_gui(window):
    global correct_streak  # 声明使用全局变量

    # 清除窗口中现有的所有组件
    for widget in window.winfo_children():
        widget.destroy()

    # 随机选取一个类别，并在该类别中选择正确答案
    class_name = random.choice(list(drugs_structures.keys()))
    correct_drug, correct_link = random.choice(list(drugs_structures[class_name].items()))

    # 从相同类别中移除正确答案，以便从剩余药物中选取错误选项
    temp_structures = drugs_structures[class_name].copy()
    del temp_structures[correct_drug]

    # 随机选择三个错误选项
    wrong_links = random.sample(list(temp_structures.values()), 3)
    all_options = wrong_links + [correct_link]
    random.shuffle(all_options)  # 打乱所有选项的顺序

    # 创建一个Label来显示药物名称
    tk.Label(window, text=f"Q：{correct_drug}", font=("Arial", 16)).pack()

    # 创建一个Frame来放置选项按钮
    frame = tk.Frame(window)
    frame.pack(pady=20)

    # 定义一个函数来处理用户的选择
    def handle_choice(index):
        global correct_streak  # 再次声明使用全局变量
        if all_options[index] == correct_link:
            correct_streak += 1
            #print("正确！")
        else:
            correct_index = all_options.index(correct_link) + 1
            messagebox.showerror("结果", f"回答错误。\n正确的结构式是选项 {correct_index}。\n你之前连续答对了 {correct_streak} 题。")
            correct_streak = 0

        if correct_streak >= target_streak:
            window.destroy()
        else:
            generate_question_gui(window)  # 生成新的题目

    # 为每个选项创建一个按钮，并附上图片
    for i, option in enumerate(all_options):
        try:
            # 加载图片
            img = Image.open(option)
            # 调整图片大小并保持长宽比，限制在600x75范围内
            img.thumbnail((600, 300), Image.Resampling.LANCZOS)
            # 创建一个新的白色背景图片
            background = Image.new('RGB', (600, 300), (255, 255, 255))
            # 计算图像放置的位置，使其居中
            img_w, img_h = img.size
            bg_w, bg_h = background.size
            offset = ((bg_w - img_w) // 2, (bg_h - img_h) // 2)
            # 将缩放后的图像粘贴到白色背景上
            background.paste(img, offset)
            img = ImageTk.PhotoImage(background)
            button = tk.Button(frame, image=img, command=lambda i=i: handle_choice(i))
            button.img = img  # 保留对图片的引用
            button.grid(row=i//2, column=i%2, padx=10, pady=10)  # 2*2布局，并添加间距
        except Exception as e:
            print(f"无法加载图片 {option}: {e}")

    # 添加一个退出按钮
    tk.Button(window, text="退出", command=window.quit, font=("Arial", 14)).pack(pady=10)

if __name__ == '__main__':
    base_folder = r'D:\desktop\Mein Kampf\Pic'  # 替换为实际的基准文件夹路径
    db_path = r'd:\desktop\Mein Kampf\Guerrilla.db'  # 替换为实际的数据库路径
    create_table(db_path)
    read_folder_and_insert_data(base_folder, db_path)
    
    # 更新药物结构字典
    data = load_drug_data(db_path)
    drugs_structures = {}
    for row in data:
        q, pathway, class_name = row
        if class_name not in drugs_structures:
            drugs_structures[class_name] = {}
        drugs_structures[class_name][q] = pathway

    # 创建图形界面窗口
    window = tk.Tk()
    window.title("药物结构式识别 Quiz")

    # 第一次生成题目
    generate_question_gui(window)

    # 运行图形界面的主循环
    window.mainloop()
# endregion

易

In [3]:
# region
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import sqlite3
import random
from collections import Counter

# 数据库路径
DB_PATH = r"d:\desktop\Mein Kampf\Guerrilla.db"

# 答对次数计数器
correct_answers_count = 0

# 按钮尺寸
BUTTON_WIDTH, BUTTON_HEIGHT = 240, 240

def fetch_questions_with_random_ratio(db_path, num_questions=5):
    """
    从数据库中随机筛选符合条件的题目
    :param db_path: str, 数据库路径
    :param num_questions: int, 需要的题目数量
    :return: list, 最终筛选的题目
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # 查询所有题目
    cursor.execute("SELECT * FROM Var")
    rows = cursor.fetchall()
    conn.close()

    # 随机生成 0 和 1 的比例
    x = random.randint(1, 5)  # 随机生成 0 的数量
    target_ratio = {0: x, 1: 6 - x}  # 目标比例

    # 筛选符合条件的题目
    filtered_rows = []
    for row in rows:
        gua_name = row[0]  # 卦名
        values = row[1:]   # 卦象数据

        # 统计当前行的数字比例
        counts = Counter(values)
        is_match = counts.get("0", 0) == target_ratio[0] and counts.get("1", 0) == target_ratio[1]

        if is_match:
            filtered_rows.append((gua_name, values))

    # 如果符合条件的题目不足，补充题目
    if len(filtered_rows) < num_questions:
        print("符合条件的题目不足，从所有题目中随机补充。")
        filtered_rows += random.sample(rows, num_questions - len(filtered_rows))

    # 从筛选结果中随机抽取题目
    selected_questions = random.sample(filtered_rows, num_questions)
    return selected_questions

def fetch_questions_with_matching_columns(db_path, num_questions=8):
    """
    从数据库中筛选出前三列和后三列完全相同的题目
    :param db_path: str, 数据库路径
    :param num_questions: int, 需要的题目数量
    :return: list, 最终筛选的题目
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # 查询所有题目
    cursor.execute("SELECT * FROM Var")
    rows = cursor.fetchall()
    conn.close()

    # 筛选符合条件的题目
    filtered_rows = []
    for row in rows:
        gua_name = row[0]  # 卦名
        values = row[1:]   # 卦象数据

        # 检查前三列和后三列是否相同
        if values[:3] == values[-3:]:
            filtered_rows.append((gua_name, values))
    
    # 如果符合条件的题目不足，补充题目
    if len(filtered_rows) < num_questions:
        print("符合条件的题目不足，从所有题目中随机补充。")
        filtered_rows += random.sample(rows, num_questions - len(filtered_rows))

    # 从筛选结果中随机抽取题目
    selected_questions = random.sample(filtered_rows, num_questions)
    return selected_questions

def generate_question_gui(window, questions):
    """
    生成 Tkinter GUI 界面，用于答题
    :param window: tk.Tk 窗口对象
    :param questions: list, 筛选出的题目
    """
    global correct_answers_count

    # 清除窗口中现有的所有组件
    for widget in window.winfo_children():
        widget.destroy()

    # 随机选择一个正确答案
    correct_question = random.choice(questions)
    correct_gua = correct_question[0]

    # 图片路径（根据卦名）
    correct_image = f"D:/desktop/64/{correct_gua}.png"

    # 准备错误选项，确保不包含正确答案
    temp_questions = [q for q in questions if q[0] != correct_gua]
    wrong_images = [f"D:/desktop/64/{q[0]}.png" for q in random.sample(temp_questions, 3)]

    # 合并正确答案和错误选项，并随机排列
    all_options = wrong_images + [correct_image]
    random.shuffle(all_options)

    # 显示问题
    tk.Label(window, text=f"Q: {correct_gua}", font=("Arial", 16)).pack()

    # 设置选项框
    frame = tk.Frame(window)
    frame.pack(pady=20)

    def handle_choice(index):
        global correct_answers_count
        if all_options[index] == correct_image:
            correct_answers_count += 1
            if correct_answers_count >= 20:
                window.destroy()
            else:
                generate_question_gui(window, questions)  # 加载下一题
        else:
            correct_answers_count = 0
            correct_index = all_options.index(correct_image) + 1
            messagebox.showerror("结果", f"答错了！\n正确图片是选项 {correct_index}。")
            generate_question_gui(window, questions)  # 重置答题

    # 为每个选项创建按钮，并附上图片
    for i, option in enumerate(all_options):
        try:
            img = Image.open(option).convert("RGBA")
            background = Image.new("RGBA", img.size, (255, 255, 255))
            combined = Image.alpha_composite(background, img)
            combined = combined.resize((BUTTON_WIDTH, BUTTON_HEIGHT), Image.Resampling.LANCZOS)
            combined = combined.convert("RGB")
            photo_img = ImageTk.PhotoImage(combined)
            button = tk.Button(frame, image=photo_img, command=lambda i=i: handle_choice(i))
            button.photo_img = photo_img
            button.grid(row=i // 2, column=i % 2, padx=10, pady=10)
        except Exception as e:
            print(f"无法加载图片 {option}: {e}")

    tk.Button(window, text="退出", command=window.quit, font=("Arial", 14)).pack()

# 主程序入口
def main():
    global correct_answers_count
    correct_answers_count = 0  # 初始化计数

    # 从数据库中获取符合条件的题目
    questions = fetch_questions_with_matching_columns(DB_PATH, num_questions=8)

    if not questions:
        print("没有足够符合条件的题目！请检查数据库数据。")
        return

    # 创建窗口
    window = tk.Tk()
    window.title("64卦象识别 Quiz")
    generate_question_gui(window, questions)
    window.mainloop()

if __name__ == "__main__":
    main()

# endregion